# ImageNet图像与类别名称匹配演示

本笔记本展示如何将ImageNet图像与其对应的类别名称进行匹配。

In [ ]:
import os
import matplotlib.pyplot as plt
from PIL import Image
from imagenet_utils import create_imagenet_metadata

# 设置中文显示
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

## 1. 加载ImageNet元数据

ImageNet使用WordNet ID (WNID)来标识类别，格式为`n########`（如`n01530575`）。

元数据文件`meta.mat`包含了WNID到类别名称的映射关系。

In [ ]:
# 创建元数据对象
DATA_ROOT = r"G:\Thomas\3_1_project\data\ImageNet-data"

print("加载ImageNet元数据...")
metadata = create_imagenet_metadata(DATA_ROOT)
print(f"✓ 成功加载 {len(metadata)} 个类别的元数据\n")

# 显示数据结构
print("数据目录结构:")
print(f"  数据根目录: {DATA_ROOT}")
print(f"  训练数据: {os.path.join(DATA_ROOT, 'train')}")
print(f"  元数据: {os.path.join(DATA_ROOT, 'meta')}")

## 2. WNID到类别名称的映射

### 2.1 查询单个WNID

In [ ]:
# 示例: 查询特定WNID的类别信息
sample_wnids = [
    "n01530575",  # brambling (燕雀)
    "n02099267",  # flat-coated retriever (平毛寻回犬)
    "n03594734",  # jean (牛仔裤)
    "n02480495",  # orangutan (猩猩)
    "n03761084",  # microwave (微波炉)
]

print("WNID到类别名称映射示例:")
print("=" * 70)
for wnid in sample_wnids:
    name = metadata.get_class_name(wnid)
    desc = metadata.get_class_description(wnid)
    print(f"WNID: {wnid}")
    print(f"  名称: {name}")
    print(f"  描述: {desc}")
    print()

### 2.2 扫描本地可用的类别

In [ ]:
# 扫描训练数据目录，查看本地有哪些类别
train_dir = os.path.join(DATA_ROOT, "train")

local_wnids = []
for item in sorted(os.listdir(train_dir)):
    item_path = os.path.join(train_dir, item)
    if os.path.isdir(item_path) and item.startswith('n'):
        local_wnids.append(item)

print(f"本地共有 {len(local_wnids)} 个类别的数据\n")
print("本地类别列表:")
print("=" * 70)
for i, wnid in enumerate(local_wnids, 1):
    name = metadata.get_class_name(wnid)
    # 统计该类别的图像数量
    class_dir = os.path.join(train_dir, wnid)
    num_images = len([f for f in os.listdir(class_dir) if f.endswith('.JPEG')])
    print(f"{i:2d}. {wnid}: {name:50s} ({num_images} 张图像)")

## 3. 从图像路径获取类别名称

ImageNet的图像文件命名规则：`{WNID}_{image_id}.JPEG`

例如：`n01530575_10007.JPEG` 表示类别`n01530575`的第10007张图像。

In [ ]:
# 获取一些示例图像路径
sample_images = []

for wnid in local_wnids[:5]:  # 前5个类别
    class_dir = os.path.join(train_dir, wnid)
    images = [f for f in os.listdir(class_dir) if f.endswith('.JPEG')][:2]  # 每个类别取2张
    for img in images:
        img_path = os.path.join(class_dir, img)
        sample_images.append(img_path)

print("图像路径到类别名称转换示例:")
print("=" * 70)
for img_path in sample_images:
    # 提取WNID
    wnid = metadata.get_wnid_from_path(img_path)
    # 获取类别名称
    class_name = metadata.get_name_from_path(img_path)
    # 相对路径（更简洁）
    rel_path = os.path.relpath(img_path, train_dir)
    
    print(f"路径: {rel_path}")
    print(f"  WNID: {wnid}")
    print(f"  类别: {class_name}")
    print()

## 4. 可视化：展示图像及其类别名称

将图像和对应的类别名称一起展示，验证匹配的正确性。

In [ ]:
def visualize_images_with_labels(image_paths, metadata, cols=4):
    """
    可视化图像及其类别标签
    
    Args:
        image_paths: 图像路径列表
        metadata: ImageNetMetadata对象
        cols: 每行显示的列数
    """
    num_images = len(image_paths)
    rows = (num_images + cols - 1) // cols
    
    fig, axes = plt.subplots(rows, cols, figsize=(cols*4, rows*4))
    axes = axes.flatten() if rows > 1 or cols > 1 else [axes]
    
    for idx, img_path in enumerate(image_paths):
        # 加载图像
        img = Image.open(img_path).convert('RGB')
        
        # 获取类别信息
        wnid = metadata.get_wnid_from_path(img_path)
        class_name = metadata.get_class_name(wnid)
        
        # 显示图像
        axes[idx].imshow(img)
        axes[idx].set_title(f"{wnid}\n{class_name}", fontsize=10, fontweight='bold')
        axes[idx].axis('off')
    
    # 隐藏多余的子图
    for idx in range(num_images, len(axes)):
        axes[idx].axis('off')
    
    plt.tight_layout()
    plt.show()

# 展示图像
print("可视化图像及其类别标签:")
print("=" * 70)
visualize_images_with_labels(sample_images[:12], metadata, cols=4)

## 5. 批量处理：创建图像-标签对列表

实际应用中，我们通常需要批量处理所有图像，生成图像路径和标签的对应关系。

In [ ]:
def create_image_label_pairs(train_dir, metadata, max_per_class=None):
    """
    创建图像路径和标签的对应关系
    
    Args:
        train_dir: 训练数据目录
        metadata: ImageNetMetadata对象
        max_per_class: 每个类别最多取多少张图像（None表示全部）
    
    Returns:
        list of tuples: [(image_path, wnid, class_name), ...]
    """
    pairs = []
    
    for wnid in sorted(os.listdir(train_dir)):
        class_dir = os.path.join(train_dir, wnid)
        if not os.path.isdir(class_dir) or not wnid.startswith('n'):
            continue
        
        # 获取类别名称
        class_name = metadata.get_class_name(wnid)
        
        # 获取该类别的所有图像
        images = [f for f in os.listdir(class_dir) if f.endswith('.JPEG')]
        
        # 限制数量
        if max_per_class is not None:
            images = images[:max_per_class]
        
        # 添加到列表
        for img in images:
            img_path = os.path.join(class_dir, img)
            pairs.append((img_path, wnid, class_name))
    
    return pairs

# 创建配对列表（每个类别最多10张）
print("创建图像-标签配对列表...")
pairs = create_image_label_pairs(train_dir, metadata, max_per_class=10)
print(f"✓ 共创建 {len(pairs)} 个配对\n")

# 显示前10个配对
print("前10个配对示例:")
print("=" * 70)
for i, (img_path, wnid, class_name) in enumerate(pairs[:10], 1):
    rel_path = os.path.relpath(img_path, train_dir)
    print(f"{i:2d}. {rel_path}")
    print(f"    WNID: {wnid}, 类别: {class_name}\n")

## 6. 保存映射到文件（可选）

可以将WNID到类别名称的映射保存为CSV或JSON文件，方便后续使用。

In [ ]:
import json
import csv

# 创建本地类别的映射字典
local_mapping = {}
for wnid in local_wnids:
    local_mapping[wnid] = {
        'class_name': metadata.get_class_name(wnid),
        'description': metadata.get_class_description(wnid)
    }

# 保存为JSON
json_path = os.path.join(DATA_ROOT, "class_mapping.json")
with open(json_path, 'w', encoding='utf-8') as f:
    json.dump(local_mapping, f, indent=2, ensure_ascii=False)
print(f"✓ 已保存JSON映射文件: {json_path}")

# 保存为CSV
csv_path = os.path.join(DATA_ROOT, "class_mapping.csv")
with open(csv_path, 'w', encoding='utf-8', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['WNID', 'Class Name', 'Description'])
    for wnid in sorted(local_wnids):
        writer.writerow([
            wnid,
            metadata.get_class_name(wnid),
            metadata.get_class_description(wnid)
        ])
print(f"✓ 已保存CSV映射文件: {csv_path}")

# 显示保存的内容（前5行）
print("\nCSV文件内容预览:")
print("=" * 70)
with open(csv_path, 'r', encoding='utf-8') as f:
    for i, line in enumerate(f):
        if i < 6:  # 显示前6行（包括标题）
            print(line.strip())

## 总结

✅ **成功建立了ImageNet图像与类别名称的映射关系！**

### 关键点：

1. **WNID格式**：`n########`（如`n01530575`）
2. **图像命名**：`{WNID}_{image_id}.JPEG`
3. **元数据文件**：`meta.mat`包含WNID到类别名称的映射
4. **类别名称**：可能包含多个同义词，用逗号分隔

### 使用方法：

```python
from imagenet_utils import create_imagenet_metadata

# 创建元数据对象
metadata = create_imagenet_metadata()

# 方法1: 通过WNID查询
class_name = metadata.get_class_name('n01530575')

# 方法2: 从图像路径直接获取
class_name = metadata.get_name_from_path('train/n01530575/n01530575_10007.JPEG')
```

### 本地数据统计：

- 总类别数: 29 个（ImageNet完整版有1000个类别）
- 元数据库: 1860 个类别（包含所有WordNet层级）
- 每个类别约有数百到上千张图像